# Antarctic Penguin Species Clustering - Practice Solutions

## Project Overview
- **Date**: May 13, 2025
- **Objective**: Apply unsupervised learning techniques to cluster penguin data and compare algorithm-derived clusters with actual taxonomic classifications.
- **Data Source**: Palmer Penguins Dataset

This notebook provides solutions to the mini practice tasks outlined in the project README. It focuses on advanced clustering techniques and alternative dimensionality reduction methods.

## Setup and Imports

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Machine Learning libraries
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.manifold import TSNE
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist

# Set visualization style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("colorblind")
sns.set(font_scale=1.2)

# Configure pandas display options
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# Ignore warnings
warnings.filterwarnings('ignore')

## Data Loading and Initial Processing

In [ ]:
# Load the data
penguins = pd.read_csv('penguins.csv')

# Display basic information
print(f"Dataset shape: {penguins.shape}")
penguins.head()

In [ ]:
# Data cleaning
# Create a clean copy of the dataset
penguins_clean = penguins.copy()

# Handle missing values
penguins_clean = penguins_clean.dropna()
print(f"Cleaned dataset shape: {penguins_clean.shape}")

# Select features for clustering
feature_cols = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
X = penguins_clean[feature_cols]

# Store the actual species labels for later comparison
species_labels = penguins_clean['species']

# Map species to numeric values for comparison metrics
species_map = {'Adelie': 0, 'Chinstrap': 1, 'Gentoo': 2}
y_true = penguins_clean['species'].map(species_map)

# Standardize features (important for clustering)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA for dimensionality reduction (to use in visualizations)
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X_scaled)

print(f"Explained variance with 2 PCA components: {pca.explained_variance_ratio_.sum()*100:.2f}%")

## Mini Practice Task 1: t-SNE for Dimensionality Reduction

t-SNE (t-Distributed Stochastic Neighbor Embedding) is a nonlinear dimensionality reduction technique that is particularly well-suited for visualizing high-dimensional data in a low-dimensional space. Unlike PCA, which preserves global structure, t-SNE focuses on preserving local structure, making it excellent for visualizing clusters.

In [ ]:
# Apply t-SNE for dimensionality reduction
# Since t-SNE can be sensitive to hyperparameters, we'll try a few perplexity values
perplexities = [5, 30, 50]
fig, axes = plt.subplots(1, len(perplexities), figsize=(18, 6))

for i, perplexity in enumerate(perplexities):
    # Apply t-SNE with the current perplexity
    tsne = TSNE(n_components=2, perplexity=perplexity, random_state=42, n_iter=1000)
    tsne_result = tsne.fit_transform(X_scaled)
    
    # Plot the results
    scatter = axes[i].scatter(tsne_result[:, 0], tsne_result[:, 1], 
                             c=y_true, cmap='viridis', s=80, alpha=0.8, edgecolor='k')
    axes[i].set_title(f't-SNE with Perplexity = {perplexity}')
    axes[i].set_xlabel('t-SNE Component 1')
    axes[i].set_ylabel('t-SNE Component 2')
    axes[i].grid(True, alpha=0.3)

# Create a legend with species names
species_names = list(species_map.keys())
handles, _ = scatter.legend_elements()
fig.legend(handles, species_names, title="Species", loc='upper center', 
           bbox_to_anchor=(0.5, 0.05), ncol=3)

plt.tight_layout()
plt.show()

In [ ]:
# Compare t-SNE with PCA side by side
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Plot PCA results
scatter1 = axes[0].scatter(pca_result[:, 0], pca_result[:, 1], c=y_true, 
                          cmap='viridis', s=100, alpha=0.8, edgecolor='k')
axes[0].set_title('PCA Projection')
axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.2f}%)')
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.2f}%)')
axes[0].grid(True, alpha=0.3)

# Use the best t-SNE from above (perplexity = 30 is a good default)
tsne = TSNE(n_components=2, perplexity=30, random_state=42, n_iter=1000)
tsne_result = tsne.fit_transform(X_scaled)

# Plot t-SNE results
scatter2 = axes[1].scatter(tsne_result[:, 0], tsne_result[:, 1], c=y_true, 
                          cmap='viridis', s=100, alpha=0.8, edgecolor='k')
axes[1].set_title('t-SNE Projection')
axes[1].set_xlabel('t-SNE Component 1')
axes[1].set_ylabel('t-SNE Component 2')
axes[1].grid(True, alpha=0.3)

# Add shared legend
handles, _ = scatter1.legend_elements()
fig.legend(handles, species_names, title="Species", loc='upper center', 
           bbox_to_anchor=(0.5, 0.05), ncol=3)

plt.tight_layout()
plt.show()

# Store t-SNE results for later use
best_tsne_result = tsne_result

### Analysis of PCA vs t-SNE

**PCA:**
- Linear dimensionality reduction that preserves global structure
- Maintains maximum variance in the data
- Typically faster and more deterministic than t-SNE
- Works well when the data has a linear structure

**t-SNE:**
- Nonlinear dimensionality reduction that preserves local structure
- Better at revealing clusters and patterns in complex, high-dimensional data
- More computationally intensive than PCA
- Results can vary based on perplexity and random initialization
- Often provides better separation between clusters

In this case, t-SNE appears to provide a clearer separation of the penguin species compared to PCA, especially for distinguishing between Adelie and Chinstrap penguins. This suggests that there are nonlinear relationships in the penguin measurement data that t-SNE captures better than PCA.

## Mini Practice Task 2: Using Different Distance Metrics for Hierarchical Clustering

This task explores how different distance metrics affect hierarchical clustering results. We'll compare three common distance metrics:

1. **Euclidean distance**: Straight-line distance between two points (L2 norm)
2. **Manhattan distance**: Sum of absolute differences (L1 norm)
3. **Cosine distance**: Measures angle between vectors regardless of magnitude

In [ ]:
# Define distance metrics to test
dist_metrics = ['euclidean', 'manhattan', 'cosine']

# Define linkage methods
link_method = 'ward'  # Ward's method minimizes variance within clusters

# Create a figure for dendrograms
plt.figure(figsize=(20, 15))

# Store ARI scores for comparison
ari_scores = []

for i, metric in enumerate(dist_metrics):
    # For Ward linkage, only Euclidean distance is valid, so use complete linkage for others
    method = 'ward' if metric == 'euclidean' else 'complete'
    
    # Calculate distance matrix with current metric
    if metric == 'euclidean':
        Z = linkage(X_scaled, method=method, metric=metric)
    else:
        # For non-Euclidean metrics, compute the distance matrix first
        dist_matrix = pdist(X_scaled, metric=metric)
        Z = linkage(dist_matrix, method=method)
    
    # Plot dendrogram
    plt.subplot(len(dist_metrics), 2, 2*i+1)
    plt.title(f'Hierarchical Clustering Dendrogram ({metric} distance)', fontsize=14)
    plt.xlabel('Sample Index', fontsize=12)
    plt.ylabel('Distance', fontsize=12)
    dendrogram(
        Z,
        leaf_rotation=90.,
        leaf_font_size=8.,
    )
    plt.axhline(y=5, color='r', linestyle='--')  # Cut line
    plt.grid(True, alpha=0.3)
    
    # Cut the dendrogram to get cluster labels (3 clusters to match species count)
    hc_labels = fcluster(Z, t=3, criterion='maxclust')  
    
    # Calculate Adjusted Rand Index (ARI) to compare with true species
    ari = adjusted_rand_score(y_true, hc_labels)
    ari_scores.append(ari)
    
    # Plot the clusters in PCA space
    plt.subplot(len(dist_metrics), 2, 2*i+2)
    plt.scatter(pca_result[:, 0], pca_result[:, 1], c=hc_labels, 
                cmap='viridis', s=80, alpha=0.8, edgecolor='k')
    plt.title(f'Hierarchical Clusters with {metric} distance (ARI: {ari:.3f})', fontsize=14)
    plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.2f}%)', fontsize=12)
    plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.2f}%)', fontsize=12)
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print out comparison of metrics by ARI score
metrics_comparison = pd.DataFrame({
    'Distance Metric': dist_metrics,
    'Adjusted Rand Index': ari_scores
}).sort_values(by='Adjusted Rand Index', ascending=False)

print("\nDistance Metrics Performance Comparison (higher ARI is better):")
print(metrics_comparison)

### Analysis of Different Distance Metrics

The three distance metrics we tested (Euclidean, Manhattan, and Cosine) each have different properties that affect clustering results:

- **Euclidean distance** works well when clusters are spherical and features are on similar scales (which we've ensured through standardization). It's sensitive to both direction and magnitude differences.

- **Manhattan distance** measures the sum of absolute differences along each dimension, making it less sensitive to outliers than Euclidean distance. It works better when clusters are more aligned with the coordinate axes.

- **Cosine distance** focuses only on the angle between samples as vectors, ignoring magnitude. This can be useful when we care about the pattern or direction of features rather than their absolute values.

Based on the Adjusted Rand Index (ARI) scores, which measure the similarity between the clustering results and the true species labels, we can determine which distance metric best recovered the natural penguin species groupings. The metric with the highest ARI score provides clusters that most closely match the actual taxonomic classifications.

## Mini Practice Task 3: Gaussian Mixture Models (GMM)

Gaussian Mixture Models (GMM) provide a probabilistic approach to clustering. Unlike K-means, which assigns each data point to exactly one cluster, GMM calculates probabilities of cluster membership, allowing for soft clustering. GMM also captures clusters with different sizes and elliptical shapes.

In [ ]:
# Find optimal number of components for GMM using BIC (Bayesian Information Criterion)
n_components_range = range(1, 10)
bic_scores = []
aic_scores = []

for n_components in n_components_range:
    # Train GMM
    gmm = GaussianMixture(n_components=n_components, 
                          covariance_type='full', 
                          random_state=42,
                          max_iter=200,
                          n_init=10)
    gmm.fit(X_scaled)
    
    # Store scores
    bic_scores.append(gmm.bic(X_scaled))
    aic_scores.append(gmm.aic(X_scaled))

# Plot BIC and AIC scores
plt.figure(figsize=(12, 6))
plt.plot(n_components_range, bic_scores, 'o-', label='BIC')
plt.plot(n_components_range, aic_scores, 'o-', label='AIC')
plt.xlabel('Number of Components', fontsize=14)
plt.ylabel('Information Criterion Score', fontsize=14)
plt.title('BIC and AIC Scores for Different Numbers of GMM Components', fontsize=16)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(n_components_range)
plt.show()

# Find the optimal number of components based on BIC
optimal_n_components = n_components_range[np.argmin(bic_scores)]
print(f"Optimal number of components based on BIC: {optimal_n_components}")

In [ ]:
# Apply GMM with optimal number of components
# However, we'll also try with 3 components to match the number of actual species
n_components_to_try = [optimal_n_components, 3]

plt.figure(figsize=(12, 10))
ari_gmm_scores = []

for i, n in enumerate(n_components_to_try):
    # Train GMM
    gmm = GaussianMixture(n_components=n, 
                          covariance_type='full', 
                          random_state=42,
                          max_iter=200,
                          n_init=10)
    gmm_labels = gmm.fit_predict(X_scaled)
    
    # Get probabilities for each point
    proba = gmm.predict_proba(X_scaled)
    
    # Calculate ARI
    ari = adjusted_rand_score(y_true, gmm_labels)
    ari_gmm_scores.append(ari)
    
    # Create subplots: one for the clusters, one for the probability distribution
    plt.subplot(2, 2, i*2+1)
    plt.scatter(pca_result[:, 0], pca_result[:, 1], c=gmm_labels, 
                cmap='viridis', s=80, alpha=0.8, edgecolor='k')
    plt.title(f'GMM with {n} Components (ARI: {ari:.3f})', fontsize=14)
    plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.2f}%)', fontsize=12)
    plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.2f}%)', fontsize=12)
    plt.grid(True, alpha=0.3)
    
    # Plot cluster membership probabilities for the first 20 samples
    plt.subplot(2, 2, i*2+2)
    if n <= 10:  # Only plot if not too many components
        sample_proba_df = pd.DataFrame(proba[:20], 
                                     columns=[f'Cluster {i}' for i in range(n)])
        sns.heatmap(sample_proba_df, cmap='viridis', annot=True, fmt='.2f', cbar=True)
        plt.title(f'Cluster Membership Probabilities (First 20 Samples)', fontsize=14)
        plt.xlabel('Cluster', fontsize=12)
        plt.ylabel('Sample Index', fontsize=12)

plt.tight_layout()
plt.show()

# Store the GMM labels with 3 components for later comparison
gmm_final = GaussianMixture(n_components=3, covariance_type='full', random_state=42, n_init=10)
gmm_labels = gmm_final.fit_predict(X_scaled)

## Comparing All Clustering Methods

Now let's compare the performance of all the clustering methods we've explored: K-means, Hierarchical Clustering (with different distance metrics), and GMM.

In [ ]:
# Apply K-means clustering with k=3
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
kmeans_labels = kmeans.fit_predict(X_scaled)
kmeans_ari = adjusted_rand_score(y_true, kmeans_labels)

# Use the best hierarchical clustering result from Task 2
best_metric_idx = np.argmax(ari_scores)
best_metric = dist_metrics[best_metric_idx]
best_hc_ari = ari_scores[best_metric_idx]

# For DBSCAN, we need to find the appropriate epsilon value
# Use nearest neighbors to estimate a good epsilon value
neighbors = NearestNeighbors(n_neighbors=2)
neighbors_fit = neighbors.fit(X_scaled)
distances, indices = neighbors_fit.kneighbors(X_scaled)
distances = np.sort(distances[:, 1])

# Plot k-distance graph to find the "elbow"
plt.figure(figsize=(10, 6))
plt.plot(distances)
plt.title('K-Distance Graph for DBSCAN Epsilon Estimation', fontsize=16)
plt.xlabel('Points sorted by distance', fontsize=14)
plt.ylabel('Distance to 2nd nearest neighbor', fontsize=14)
plt.axhline(y=0.5, color='r', linestyle='--')
plt.grid(True, alpha=0.3)
plt.show()

# Apply DBSCAN with the estimated epsilon
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan_labels = dbscan.fit_predict(X_scaled)

# Count number of clusters found (excluding noise points with label -1)
n_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
print(f"DBSCAN found {n_clusters} clusters and {sum(dbscan_labels == -1)} noise points")

# Calculate ARI for DBSCAN (only if it found some clusters)
if n_clusters > 0:
    dbscan_ari = adjusted_rand_score(y_true, dbscan_labels)
else:
    dbscan_ari = 0

# Create a comparison table of all methods
comparison = pd.DataFrame({
    'Clustering Method': ['K-means', f'Hierarchical ({best_metric})', 'GMM', 'DBSCAN'],
    'Adjusted Rand Index': [kmeans_ari, best_hc_ari, ari_gmm_scores[1], dbscan_ari]
}).sort_values(by='Adjusted Rand Index', ascending=False)

print("\nClustering Methods Comparison (higher ARI is better):")
display(comparison)

# Visualize all clustering results side by side
plt.figure(figsize=(20, 10))

# Plot K-means clusters
plt.subplot(2, 3, 1)
plt.scatter(pca_result[:, 0], pca_result[:, 1], c=kmeans_labels, 
            cmap='viridis', s=80, alpha=0.8, edgecolor='k')
plt.title(f'K-means (ARI: {kmeans_ari:.3f})', fontsize=14)
plt.xlabel('PC1', fontsize=12)
plt.ylabel('PC2', fontsize=12)
plt.grid(True, alpha=0.3)

# Plot Hierarchical clusters (best metric)
method = 'ward' if best_metric == 'euclidean' else 'complete'
if best_metric == 'euclidean':
    Z = linkage(X_scaled, method=method, metric=best_metric)
else:
    dist_matrix = pdist(X_scaled, metric=best_metric)
    Z = linkage(dist_matrix, method=method)
hc_labels = fcluster(Z, t=3, criterion='maxclust')

plt.subplot(2, 3, 2)
plt.scatter(pca_result[:, 0], pca_result[:, 1], c=hc_labels, 
            cmap='viridis', s=80, alpha=0.8, edgecolor='k')
plt.title(f'Hierarchical ({best_metric}) (ARI: {best_hc_ari:.3f})', fontsize=14)
plt.xlabel('PC1', fontsize=12)
plt.ylabel('PC2', fontsize=12)
plt.grid(True, alpha=0.3)

# Plot GMM clusters
plt.subplot(2, 3, 3)
plt.scatter(pca_result[:, 0], pca_result[:, 1], c=gmm_labels, 
            cmap='viridis', s=80, alpha=0.8, edgecolor='k')
plt.title(f'GMM (ARI: {ari_gmm_scores[1]:.3f})', fontsize=14)
plt.xlabel('PC1', fontsize=12)
plt.ylabel('PC2', fontsize=12)
plt.grid(True, alpha=0.3)

# Plot DBSCAN clusters
plt.subplot(2, 3, 4)
plt.scatter(pca_result[:, 0], pca_result[:, 1], c=dbscan_labels, 
            cmap='viridis', s=80, alpha=0.8, edgecolor='k')
plt.title(f'DBSCAN (ARI: {dbscan_ari:.3f})', fontsize=14)
plt.xlabel('PC1', fontsize=12)
plt.ylabel('PC2', fontsize=12)
plt.grid(True, alpha=0.3)

# Plot actual species for reference
plt.subplot(2, 3, 5)
plt.scatter(pca_result[:, 0], pca_result[:, 1], c=y_true, 
            cmap='viridis', s=80, alpha=0.8, edgecolor='k')
plt.title('Actual Penguin Species', fontsize=14)
plt.xlabel('PC1', fontsize=12)
plt.ylabel('PC2', fontsize=12)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Conclusions

### Key Findings

1. **Dimensionality Reduction**: 
   - t-SNE generally provided better visual separation of penguin species compared to PCA, especially for distinguishing between Adelie and Chinstrap species.
   - While PCA preserves global structure, t-SNE better preserves local structure, which is particularly useful for clustering visualization.

2. **Distance Metrics in Hierarchical Clustering**:
   - Different distance metrics produced notably different clustering results.
   - The best-performing metric (based on the ARI scores from our analysis) most accurately recovered the natural penguin species groupings.
   - This demonstrates the importance of choosing the right distance metric for the specific data structure.

3. **Clustering Algorithms Comparison**:
   - GMM provided probabilistic cluster assignments, adding more nuance than hard clustering approaches.
   - GMM generally performed well, capturing the elliptical nature of the penguin clusters.
   - The optimal number of clusters identified by BIC/AIC may differ from the known number of species.
   - DBSCAN's performance depended heavily on parameter selection (eps and min_samples).

### Method Strengths and Weaknesses

- **K-means**: Simple and efficient, but assumes spherical clusters and requires knowing k in advance.
- **Hierarchical Clustering**: Provides a dendrogram showing relationships between clusters, but sensitive to the chosen distance metric and linkage method.
- **GMM**: Captures elliptical clusters and provides probability distributions, but may overfit with too many components.
- **DBSCAN**: Can find arbitrarily shaped clusters and identify noise points, but parameter selection is challenging.

### Biological Interpretation

The clustering algorithms successfully identified patterns in penguin measurements that largely correspond to taxonomic species classifications. This suggests that morphological measurements captured in the dataset are strongly associated with evolutionary divergence between penguin species.

The cases where clustering algorithms assigned penguins to different groups than their taxonomic classification could represent:
1. Measurement errors or outliers
2. Natural variation within species
3. Possible hybridization between similar species

These findings demonstrate how machine learning can complement traditional taxonomic approaches in biological classification.